# Simulation processing: step 1

Goal: take the simulated hadrons and create an output file for fj clustering

In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import vector

In [3]:
simulation_dir = '/global/cfs/cdirs/m3246/ewitkowski/cmssim'

muon_prefix = "recoMuons_muons__RECO_obj"
muon_vars = ["pfIsolationR03__sumChargedHadronPt", "pfIsolationR03__sumChargedHadronPt", "pfIsolationR03__sumNeutralHadronEt",
             "pfIsolationR03__sumPhotonEt", "pfIsolationR03__sumPUPt", "pfIsolationR04__sumPhotonEt", "pfIsolationR04__sumPUPt",
             "pfIsolationR04__sumChargedHadronPt", "pfIsolationR04__sumNeutralHadronEt"]

tower_prefix = "recoPFCandidates_particleFlow__RECO_obj"
tower_vars = ["eta_", "pdgId_", "phi_", "pt_"]


In [4]:
muon_iso_dict = {}
tower_dict = {}

for muon_var in muon_vars:
    muon_iso_dict[muon_var] = h5py.File(f'{simulation_dir}/{muon_prefix}_{muon_var}.h5', 'r')['values']

for tower_var in tower_vars:
    tower_dict[tower_var] = h5py.File(f'{simulation_dir}/{tower_prefix}_{tower_var}.h5', 'r')['values']


In [5]:
muon_mass = 0.1056583755 # GeV
particles_to_fastjet = [22, 130, 211, 1, 2]


start_read, stop_read, chunk_size = 0, 10, 10
update_freq = 1#int(chunk_size/10.0)

delta_R_isos = [0.3, 0.5]



In [10]:
current_chunk_start = start_read

while current_chunk_start < stop_read:
    
    current_chunk_stop = current_chunk_start + chunk_size
    print(f"Processing chunk from {current_chunk_start} to {current_chunk_stop}...")

    outfile_dimuons = f"/global/cfs/cdirs/m3246/rmastand/dimuonAD/data_post_fj/muons_only_{current_chunk_start}_{current_chunk_stop}_cmssim.dat"
    outfile_dimuons_isolation = f"/global/cfs/cdirs/m3246/rmastand/dimuonAD/data_post_fj/muons_iso_{current_chunk_start}_{current_chunk_stop}_cmssim.dat"
    outfile_hadrons = f"/global/cfs/cdirs/m3246/rmastand/dimuonAD/data_pre_fj/hadrons_only_{current_chunk_start}_{current_chunk_stop}_cmssim.dat"

    ofile_muons = open(outfile_dimuons, "w")
    ofile_iso = open(outfile_dimuons_isolation, "w")
    ofile_hadrons = open(outfile_hadrons, "w")
    num_rejects = 0
   
    for event in range(current_chunk_start, current_chunk_stop):
        print()
        
        loc_tower, loc_muon_iso = {}, {}

        if event % update_freq == 0:
            print(f"On event {event}...")

        # get the nonmuons
        loc_particle_exists = (tower_dict["pt_"][event] != 0)
        
        for tower_var in tower_vars:
            loc_tower[tower_var] = tower_dict[tower_var][event][loc_particle_exists]
        for muon_var in muon_vars:
            
            nonzero_inds = muon_iso_dict[muon_var][event] != 0
            
            
        # check that there is at least one muon and one antimuon
        num_muons = np.sum(loc_tower["pdgId_"]==13)
        num_amuons = np.sum(loc_tower["pdgId_"]==-13)
        
        if (num_muons < 1) or (num_amuons < 1):
            num_rejects += 1
            
        else:
            # write out the event
            
            # find the hardest muon / antimuon
            loc_mu_pt = loc_tower["pt_"][loc_tower["pdgId_"]==13]
            max_muon_pt = np.max(loc_mu_pt)
            hardest_muon_id = np.where(loc_tower["pt_"]==max_muon_pt)[0][0]
            
            loc_amu_pt = loc_tower["pt_"][loc_tower["pdgId_"]==-13]
            max_amuon_pt = np.max(loc_amu_pt)
            hardest_amuon_id = np.where(loc_tower["pt_"]==max_amuon_pt)[0][0]
      
            
            # sanity check -- just in case there are 2 particles with the same pt...
            if (loc_tower["pdgId_"][hardest_muon_id] != 13) or (loc_tower["pdgId_"][hardest_amuon_id] != -13):
                print("Something very wrong")
                exit()
            
            
            ofile_muons.write("#BEGIN\n")
            ofile_iso.write("#BEGIN\n")
            ofile_hadrons.write("#BEGIN\n")

            # get the muons
            # construct the muon 4-vector
            mu_1 = vector.obj(pt = loc_tower["pt_"][hardest_muon_id], eta = loc_tower["eta_"][hardest_muon_id], phi = loc_tower["phi_"][hardest_muon_id], M = muon_mass)
            mu_2 = vector.obj(pt = loc_tower["pt_"][hardest_amuon_id], eta = loc_tower["eta_"][hardest_amuon_id], phi = loc_tower["phi_"][hardest_amuon_id], M = muon_mass)
            dimu_system = mu_1 + mu_2

            ofile_muons.write(f"{dimu_system.pt} {dimu_system.eta} {dimu_system.phi} {dimu_system.M}\n")

            # get the hadrons and calculate muon isolation
            isolations_mu = {R:0 for R in delta_R_isos}
            isolations_amu = {R:0 for R in delta_R_isos}

            for particle_i in range(len(loc_tower["pt_"])):
                if np.abs(loc_tower["pdgId_"][particle_i]) in particles_to_fastjet:
                                        
                    # write out the particle
                    particle_vector = vector.obj(pt = loc_tower["pt_"][particle_i], eta = loc_tower["eta_"][particle_i], phi = loc_tower["phi_"][particle_i], M = 0)
                    ofile_hadrons.write(f"{particle_vector.px} {particle_vector.py} {particle_vector.pz} {particle_vector.E}\n")
                    
                    # calculate the isolation contribution to the hardest (a)muon
                    delta_R_mu = mu_1.deltaR(particle_vector)
                    delta_R_amu = mu_2.deltaR(particle_vector)
                    
                    for R in delta_R_isos:
                        if delta_R_mu <= R:
                            isolations_mu[R] += (particle_vector.pt)/(mu_1.pt)
                        if delta_R_amu <= R:
                            isolations_amu[R] += (particle_vector.pt)/(mu_2.pt)
                            
            iso_muons_line = ""
            for R in delta_R_isos:
                iso_muons_line += str(isolations_mu[R]) + " " 
            for R in delta_R_isos:
                iso_muons_line += str(isolations_amu[R]) + " " 
                   
            ofile_iso.write(f"{iso_muons_line}\n")

            ofile_muons.write("#END\n")
            ofile_iso.write("#END\n")
            ofile_hadrons.write("#END\n")


    ofile_muons.close()  
    ofile_iso.close() 
    ofile_hadrons.close()
    
    print(f"Done processing chunk.")
    print(f"{num_rejects} events without 2 muons")
    print("\n")
    
    current_chunk_start += chunk_size

print("Done completely!")



                                         

Processing chunk from 0 to 10...

On event 0...

On event 1...
{0.3: 0.028996143024414778, 0.5: 0.19552200799807906} {0.3: 0.1277649961411953, 0.5: 0.26710770558565855}

On event 2...
{0.3: 0.2059225649572909, 0.5: 0.5012140003964305} {0.3: 0.10496728401631117, 0.5: 0.20582806458696723}

On event 3...
{0.3: 0.16212590923532844, 0.5: 0.7676064022816718} {0.3: 0.28338183276355267, 0.5: 1.4304003743454814}

On event 4...

On event 5...

On event 6...
{0.3: 0.0907386657781899, 0.5: 0.32964917831122875} {0.3: 0.17405775468796492, 0.5: 0.2846010704524815}

On event 7...
{0.3: 0.20429018326103687, 0.5: 0.5380737148225307} {0.3: 0.10609580762684345, 0.5: 0.23404359281994402}

On event 8...

On event 9...
{0.3: 0.32637605257332325, 0.5: 1.3228085841983557} {0.3: 0.4270288459956646, 0.5: 1.152339680120349}
Done processing chunk.
4 events without 2 muons


Done completely!
